# Atividade: Perceptron para classificação de flores
**INSTITUTO FEDERAL DE MINAS GERIAS**
*Departamento de Engenharia e Computação*

**Professor:** Ciniro Nametala

**Aluno:** Euler Gomes


In [159]:
from operator import truediv

import pandas as pd
import numpy as np
import cupy as cp
import seaborn as sns
import pygwalker as pyg
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import plotly.graph_objects as go

In [160]:
#funcoes
#funcao degrau bipolar
def degrau_bipolar(x):
    if x > 0:
        return 1
    elif x == 0:
        return 0
    elif x < 0:
        return -1
    return None

#funcao de previsao


# Implementação do Perceptron

## Passo 1

In [161]:
#leitura do dataset
data = pd.read_csv('data/dataset_iris.csv', sep=';', decimal=',')

#analise exploratoria
#walker = pyg.walk(data)

data['x1'] = pd.to_numeric(data['x1'], errors='coerce')
data['x2'] = pd.to_numeric(data['x2'], errors='coerce')
data['x3'] = pd.to_numeric(data['x3'], errors='coerce')
data['y'] = pd.to_numeric(data['y'], errors='coerce')

data.head()

,x1,x2,x3,y
0,-0.6508,0.1097,4.0009,-1.0
1,-1.4492,0.8896,4.4005,-1.0
2,2.0850,0.6876,12.0710,-1.0
3,0.2626,1.1476,7.7985,1.0
4,0.6418,1.0234,7.0427,1.0


In [162]:
#quatidade de elementos na amostra
n_amostras = data.shape[0]

#quantidade de variaveis de entrada (subtrair a coluna de tipos)
n_variaveis = data.shape[1] -1

#separando os dados contendo apenas as variaveis de entrada
x = data.iloc[:, 0:n_variaveis].values
print(x)

[[-0.6508  0.1097  4.0009]
 [-1.4492  0.8896  4.4005]
 [ 2.085   0.6876 12.071 ]
 [ 0.2626  1.1476  7.7985]
 [ 0.6418  1.0234  7.0427]
 [ 0.2569  0.673   8.3265]
 [ 1.1155  0.6043  7.4446]
 [ 0.0914  0.3399  7.0677]
 [ 0.0121  0.5256  4.6316]
 [-0.0429  0.466   5.4323]
 [ 0.434   0.687   8.2287]
 [ 0.2735  1.0287  7.1934]
 [ 0.4839  0.4851  7.485 ]
 [ 0.4089 -0.1267  5.5019]
 [ 1.4391  0.1614  8.5843]
 [-0.9115 -0.1973  2.1962]
 [ 0.3654  1.0475  7.4858]
 [ 0.2144  0.7515  7.1699]
 [ 0.2013  1.0014  6.5489]
 [ 0.6483  0.2183  5.8991]
 [-0.1147  0.2242  7.2435]
 [-0.797   0.8795  3.8762]
 [-1.0625  0.6366  2.4707]
 [ 0.5307  0.1285  5.6883]
 [-1.22    0.7777  1.7252]
 [ 0.3957  0.1076  5.6623]
 [-0.1013  0.5989  7.1812]
 [ 2.4482  0.9455 11.2095]
 [ 2.0149  0.6192 10.9263]
 [ 0.2012  0.2611  5.4631]]


In [163]:
#inserindo o bias
bias = np.ones((n_amostras, 1)) * -1

#concatenar o bias com o dataset x
x = np.hstack((x, bias))
print(x)

[[-0.6508  0.1097  4.0009 -1.    ]
 [-1.4492  0.8896  4.4005 -1.    ]
 [ 2.085   0.6876 12.071  -1.    ]
 [ 0.2626  1.1476  7.7985 -1.    ]
 [ 0.6418  1.0234  7.0427 -1.    ]
 [ 0.2569  0.673   8.3265 -1.    ]
 [ 1.1155  0.6043  7.4446 -1.    ]
 [ 0.0914  0.3399  7.0677 -1.    ]
 [ 0.0121  0.5256  4.6316 -1.    ]
 [-0.0429  0.466   5.4323 -1.    ]
 [ 0.434   0.687   8.2287 -1.    ]
 [ 0.2735  1.0287  7.1934 -1.    ]
 [ 0.4839  0.4851  7.485  -1.    ]
 [ 0.4089 -0.1267  5.5019 -1.    ]
 [ 1.4391  0.1614  8.5843 -1.    ]
 [-0.9115 -0.1973  2.1962 -1.    ]
 [ 0.3654  1.0475  7.4858 -1.    ]
 [ 0.2144  0.7515  7.1699 -1.    ]
 [ 0.2013  1.0014  6.5489 -1.    ]
 [ 0.6483  0.2183  5.8991 -1.    ]
 [-0.1147  0.2242  7.2435 -1.    ]
 [-0.797   0.8795  3.8762 -1.    ]
 [-1.0625  0.6366  2.4707 -1.    ]
 [ 0.5307  0.1285  5.6883 -1.    ]
 [-1.22    0.7777  1.7252 -1.    ]
 [ 0.3957  0.1076  5.6623 -1.    ]
 [-0.1013  0.5989  7.1812 -1.    ]
 [ 2.4482  0.9455 11.2095 -1.    ]
 [ 2.0149  0.6192 10

## Passo 2

In [164]:
#associar cada entrada xi com cada saída y
y = data.iloc[:, n_variaveis].values
print(y)

#contar quantas amostras de cada classe existem no dataset
print(f"-1: {(y == -1).sum()}")
print(f"1: {(y == 1).sum()}")

[-1. -1. -1.  1.  1. -1.  1. -1.  1.  1. -1.  1. -1. -1. -1. -1.  1.  1.
  1.  1. -1.  1.  1.  1.  1. -1. -1.  1. -1.  1.]
-1: 14
1: 16


## Passo 3

In [165]:
#inicializar o vetor de pesos W com valores aleatorios
w = np.random.uniform(-1, 1, n_variaveis + 1)
print(w)

[ 0.92539616  0.0834397  -0.45633562  0.60435723]


## Passo 4 - 10

In [166]:
#Passo 4: definir a taxa e aprendizagem eta
eta = 0.01

#Passo 5: calcular o número de amostras em x
# já foi feito e está em n_amostras

#Passo 6: definir a tolerancia ao erro
tolerancia = 0.01

#Passo 7: setar o erro inicial maior que a tolerancia
erro_medio = tolerancia + 1

#Passo 8: criar o vetor que ira armazenar o erro de todas as epocas, epoca a epoca
erros_epoca =[]

#Passo 9: inicializa o contador de épocas
epocas = 0

#Passo 10: define o maximo de épocas
max_epocas = 100


## Passo 11 - 12

In [169]:
#Passo 11 e 12: seta criterios de parada
criterio_erro = True
criterio_epocas = True

#Passo 13: enquanto o erro for maior que a tolerancia e max_epocas não for atingido, continue o treino
while criterio_erro == True and criterio_epocas == True:
    erro_medio = 0
    epocas += epocas
    np.random.shuffle(x)
    print(x)


[[ 2.085   0.6876 12.071  -1.    ]
 [ 0.2626  1.1476  7.7985 -1.    ]
 [ 0.4839  0.4851  7.485  -1.    ]
 [-0.1147  0.2242  7.2435 -1.    ]
 [ 0.0121  0.5256  4.6316 -1.    ]
 [ 0.2012  0.2611  5.4631 -1.    ]
 [-0.0429  0.466   5.4323 -1.    ]
 [ 1.1155  0.6043  7.4446 -1.    ]
 [ 0.6483  0.2183  5.8991 -1.    ]
 [ 2.0149  0.6192 10.9263 -1.    ]
 [ 0.2735  1.0287  7.1934 -1.    ]
 [ 0.6418  1.0234  7.0427 -1.    ]
 [ 2.4482  0.9455 11.2095 -1.    ]
 [ 0.5307  0.1285  5.6883 -1.    ]
 [-0.797   0.8795  3.8762 -1.    ]
 [ 0.0914  0.3399  7.0677 -1.    ]
 [ 0.434   0.687   8.2287 -1.    ]
 [-0.6508  0.1097  4.0009 -1.    ]
 [ 0.2569  0.673   8.3265 -1.    ]
 [ 0.2013  1.0014  6.5489 -1.    ]
 [ 0.2144  0.7515  7.1699 -1.    ]
 [-1.4492  0.8896  4.4005 -1.    ]
 [ 0.3957  0.1076  5.6623 -1.    ]
 [-1.22    0.7777  1.7252 -1.    ]
 [ 0.4089 -0.1267  5.5019 -1.    ]
 [-0.1013  0.5989  7.1812 -1.    ]
 [-1.0625  0.6366  2.4707 -1.    ]
 [-0.9115 -0.1973  2.1962 -1.    ]
 [ 1.4391  0.1614  8

KeyboardInterrupt: 